In [1]:
from mainnet_launch.autopool_diagnostics.compute_rebalance_cost import (
    get_data,
    AUTO_LRT,
    BAL_ETH,
    AUTO_ETH,
    eth_client,
    AutopoolConstants,
    _fetch_all_rebalance_events
)
import pandas as pd
import plotly.express as px

RebalanceBetweenDestinations, RebalanceToIdle, SuccessfulRebalanceBetweenDestinations = _fetch_all_rebalance_events(AUTO_LRT)
rebalance_between_destinations_df, validated_spot_price_df, UnderlyingDeposited_df, UnderlyingWithdraw_df = get_data(AUTO_LRT)
# UnderlyingDeposited_df

2024-11-13 20:41:01.512 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-11-13 20:41:01.516 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-11-13 20:41:01.521 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-11-13 20:41:01.522 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-11-13 20:41:01.524 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-11-13 20:41:01.525 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-11-13 20:41:01.527 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-11-13 20:41:01.528 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-11-

In [64]:
valid_deposit_hashes = UnderlyingDeposited_df['hash'].value_counts()[UnderlyingDeposited_df['hash'].value_counts() == 1].index
valid_underlying_deposited_df=UnderlyingDeposited_df[UnderlyingDeposited_df['hash'].isin(valid_deposit_hashes)].copy()
valid_underlying_deposited_df['amountIn'] = valid_underlying_deposited_df['amount'] / 1e18
df = pd.merge(rebalance_between_destinations_df, valid_underlying_deposited_df, on='hash', how='left')
df.columns

Index(['valueStats', 'params', 'outSummaryStats', 'inSummaryStats',
       'swapOffsetPeriod', 'predictedAnnualizedGain', 'event_x', 'block_x',
       'transaction_index_x', 'log_index_x', 'hash', 'outDestinationVault',
       'inDestinationVault', 'tokenIn', 'tokenOut', 'amountOut', 'swapCost',
       'amount', 'sender', 'event_y', 'block_y', 'transaction_index_y',
       'log_index_y', 'contract_address', 'amountIn'],
      dtype='object')

In [66]:

df[['contract_address', 'amountIn', 'amountOut', 'inDestinationVault', 'outDestinationVault']]

,contract_address,amountIn,amountOut,inDestinationVault,outDestinationVault
0,0x777FAf85c8E5FC6f4332E56B989C5C94201A273C,100.512375,103.024424,0x777FAf85c8E5FC6f4332E56B989C5C94201A273C,0xE800e3760FC20aA98c5df6A9816147f190455AF3
1,0x4E12227b350E8f8fEEc41A58D36cE2fB2e2d4575,103.715831,105.124100,0x4E12227b350E8f8fEEc41A58D36cE2fB2e2d4575,0xE800e3760FC20aA98c5df6A9816147f190455AF3
2,0x2F7e096a400ded5D02762120b39A3aA4ABA072a4,165.884649,168.718684,0x2F7e096a400ded5D02762120b39A3aA4ABA072a4,0xE800e3760FC20aA98c5df6A9816147f190455AF3
3,0x4E12227b350E8f8fEEc41A58D36cE2fB2e2d4575,188.896729,191.505927,0x4E12227b350E8f8fEEc41A58D36cE2fB2e2d4575,0xE800e3760FC20aA98c5df6A9816147f190455AF3
4,0x2F7e096a400ded5D02762120b39A3aA4ABA072a4,141.824778,144.375381,0x2F7e096a400ded5D02762120b39A3aA4ABA072a4,0xE800e3760FC20aA98c5df6A9816147f190455AF3
...,...,...,...,...,...
78,0xf9779aEF9f77e78C857CB4A068c65CcBee25BAAc,98.265258,100.000000,0xf9779aEF9f77e78C857CB4A068c65CcBee25BAAc,0xE800e3760FC20aA98c5df6A9816147f190455AF3
79,0xf9779aEF9f77e78C857CB4A068c65CcBee25BAAc,98.280908,100.000000,0xf9779aEF9f77e78C857CB4A068c65CcBee25BAAc,0xE800e3760FC20aA98c5df6A9816147f190455AF3
80,0xf9779aEF9f77e78C857CB4A068c65CcBee25BAAc,97.770709,99.483984,0xf9779aEF9f77e78C857CB4A068c65CcBee25BAAc,0xE800e3760FC20aA98c5df6A9816147f190455AF3
81,0xf9779aEF9f77e78C857CB4A068c65CcBee25BAAc,73.300808,74.665379,0xf9779aEF9f77e78C857CB4A068c65CcBee25BAAc,0xE800e3760FC20aA98c5df6A9816147f190455AF3


In [55]:
hash_to_in_destination_vault = {h:d for h, d in zip(rebalance_between_destinations_df['hash'], rebalance_between_destinations_df['inDestinationVault'])}
UnderlyingWithdraw_df['hash'].apply(lambda x :x in hash_to_in_destination_vault)

0    False
0     True
1     True
0     True
1     True
0     True
1     True
2     True
3     True
0    False
0    False
0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
0    False
1    False
2    False
3    False
4    False
0    False
1    False
2    False
0     True
1     True
Name: hash, dtype: bool

In [51]:
rebalance_between_destinations_df[['hash', 'inDestinationVault']].to_dict(orient='records')

[{'hash': '0x5a6f3a2239134ef903468d88abce973e6b71e233af04ac0ee42651291e045294',
  'inDestinationVault': '0x777FAf85c8E5FC6f4332E56B989C5C94201A273C'},
 {'hash': '0x46cfba2d4f806b97e71d9465ae05556bc41cb3635c5d133c421271a09b91de16',
  'inDestinationVault': '0x4E12227b350E8f8fEEc41A58D36cE2fB2e2d4575'},
 {'hash': '0xa0ea869df690411f31e8a430670cfaea2b69a584971567945e5d38b32d89f062',
  'inDestinationVault': '0x2F7e096a400ded5D02762120b39A3aA4ABA072a4'},
 {'hash': '0xca793ddcca9f1a355bf52d55d397068f49b8c43de7998f804831daa3e9d48574',
  'inDestinationVault': '0x4E12227b350E8f8fEEc41A58D36cE2fB2e2d4575'},
 {'hash': '0x6ce994c065fa58c7b58d215373dcb7480aba639650136214a4a17ee64a3e595a',
  'inDestinationVault': '0x2F7e096a400ded5D02762120b39A3aA4ABA072a4'},
 {'hash': '0xe858c758de4f7238f6a40a7c7c6e807b876fe55a4a0ed26056bd0cfadae11f87',
  'inDestinationVault': '0x777FAf85c8E5FC6f4332E56B989C5C94201A273C'},
 {'hash': '0x63360390c04f91d79a132ccb6f9e7f82783b67d4cfd508f0fc4820b27b31bc3e',
  'inDestinati

In [50]:
{rebalance_between_destinations_df[['hash', 'inDestinationVault']].to_dict(orient='records')}

TypeError: unhashable type: 'list'

In [42]:
rebalance_between_destinations_df['inDestinationVault']

0     0x777FAf85c8E5FC6f4332E56B989C5C94201A273C
1     0x4E12227b350E8f8fEEc41A58D36cE2fB2e2d4575
2     0x2F7e096a400ded5D02762120b39A3aA4ABA072a4
3     0x4E12227b350E8f8fEEc41A58D36cE2fB2e2d4575
4     0x2F7e096a400ded5D02762120b39A3aA4ABA072a4
                         ...                    
78    0xf9779aEF9f77e78C857CB4A068c65CcBee25BAAc
79    0xf9779aEF9f77e78C857CB4A068c65CcBee25BAAc
80    0xf9779aEF9f77e78C857CB4A068c65CcBee25BAAc
81    0xf9779aEF9f77e78C857CB4A068c65CcBee25BAAc
82    0x40219bBda953ca811d2D0168Dc806a96b84791d9
Name: inDestinationVault, Length: 83, dtype: object

In [37]:
SuccessfulRebalanceBetweenDestinations

,destinationOut,lastRebalanceTimestamp,lastTimestampAddedToDestination,swapCostOffsetPeriod,event,block,transaction_index,log_index,hash
0,0x777FAf85c8E5FC6f4332E56B989C5C94201A273C,1727872307,1727785907,50,SuccessfulRebalanceBetweenDestinations,20877834,5,120,0x06c4340516aa6dd415463aff3de7ddfcd3a57bf47534...
1,0x2F7e096a400ded5D02762120b39A3aA4ABA072a4,1728045203,1726766171,50,SuccessfulRebalanceBetweenDestinations,20892184,18,120,0x786fd48cba89585f971c6e7b38a9acfed66b8226b366...
2,0x2F7e096a400ded5D02762120b39A3aA4ABA072a4,1728088343,1726766171,50,SuccessfulRebalanceBetweenDestinations,20895765,19,176,0xd2e9299ee0bcfe3ca8b69d3190e7927e549a524cbea6...
3,0x148Ca723BefeA7b021C399413b8b7426A4701500,1728502367,1728012695,50,SuccessfulRebalanceBetweenDestinations,20930151,1,75,0x084dfee5800acd5bda5e48485e68e87c015fca770376...
4,0x777FAf85c8E5FC6f4332E56B989C5C94201A273C,1728614975,1728574271,50,SuccessfulRebalanceBetweenDestinations,20939503,106,368,0x0928a563d0ab58ac4a98bf23d71c49b025a88ba173d0...
5,0x2F7e096a400ded5D02762120b39A3aA4ABA072a4,1728701807,1726766171,50,SuccessfulRebalanceBetweenDestinations,20946688,9,161,0xbe9e2919f268121cc55f8466e7af702b6c0dd312134e...
6,0x4E12227b350E8f8fEEc41A58D36cE2fB2e2d4575,1728739883,1728649919,50,SuccessfulRebalanceBetweenDestinations,20949834,22,222,0xa9b7e57b8202064b6fd48accbcfea54cb08fbb362638...
7,0x5c6aeb9ef0d5BbA4E6691f381003503FD0D45126,1729027895,1728614975,50,SuccessfulRebalanceBetweenDestinations,20973663,1,99,0x44d352511569a90d5afa007897aa64defdc53ff3f792...
8,0x5c6aeb9ef0d5BbA4E6691f381003503FD0D45126,1729179095,1728614975,50,SuccessfulRebalanceBetweenDestinations,20986197,55,375,0x0146005b9255612d4e769a90cedf8f6deb448cfea969...
9,0x5c6aeb9ef0d5BbA4E6691f381003503FD0D45126,1729211495,1728614975,50,SuccessfulRebalanceBetweenDestinations,20988882,52,231,0x88777fbc89b0309258567901c543be255fd2613f34c0...


In [40]:
rebalance_between_destinations_df[rebalance_between_destinations_df['hash'].isin(SuccessfulRebalanceBetweenDestinations['hash'])].shape

(16, 17)

In [35]:


valid_underlying_deposited_df[valid_underlying_deposited_df['block'] > 20722910]


,amount,sender,event,block,transaction_index,log_index,hash,contract_address
0,45275633771585782477,0xE800e3760FC20aA98c5df6A9816147f190455AF3,UnderlyingDeposited,21101723,5,75,0x80d0938d7a921128c6c03a5e445601f1521cd63c49e9...,0xC9b5D82652a1C8214b0971A004983d0EEeDD751C
1,33950989078690998560,0xE800e3760FC20aA98c5df6A9816147f190455AF3,UnderlyingDeposited,21104953,1,56,0xede5866db4be0f6de1e525a9b1b0235cff05ba297f4b...,0xC9b5D82652a1C8214b0971A004983d0EEeDD751C
2,56367497684686915345,0xE800e3760FC20aA98c5df6A9816147f190455AF3,UnderlyingDeposited,21122345,5,58,0x7ce685385887645bb3976ff021864dc504dd36428646...,0xC9b5D82652a1C8214b0971A004983d0EEeDD751C
3,42291495751400472670,0xE800e3760FC20aA98c5df6A9816147f190455AF3,UnderlyingDeposited,21125641,6,66,0x350476010ae54e6882868908e282c799aa1f0d6c4c54...,0xC9b5D82652a1C8214b0971A004983d0EEeDD751C
0,26412460747973757896,0xE800e3760FC20aA98c5df6A9816147f190455AF3,UnderlyingDeposited,20898452,0,45,0xb9ffffb7aa2f8aaa490f1ab6fb371166f11c6345c4dc...,0x90300b02b162F902B9629963830BcCCdeEd71113
...,...,...,...,...,...,...,...,...
10,93721328205063299580,0xE800e3760FC20aA98c5df6A9816147f190455AF3,UnderlyingDeposited,20889481,143,309,0xbafdc2344e3cfca7345d0f99a2340119d9a562c06e63...,0x148Ca723BefeA7b021C399413b8b7426A4701500
11,57795772975127563663,0x6dC3ce9C57b20131347FDc9089D740DAf6eB34c5,UnderlyingDeposited,20892779,38,233,0xf1a9200b85619fd39e358bf3fd8b993099e4a585735a...,0x148Ca723BefeA7b021C399413b8b7426A4701500
12,39697263089877165819,0x6dC3ce9C57b20131347FDc9089D740DAf6eB34c5,UnderlyingDeposited,20898160,2,51,0x3f09f5bc24ed2681ca8fe47f3e5f1510cbe6b4d808b7...,0x148Ca723BefeA7b021C399413b8b7426A4701500
13,74954049209195993609,0x6dC3ce9C57b20131347FDc9089D740DAf6eB34c5,UnderlyingDeposited,20912516,32,161,0x7845ffdf88a504a998b1caecf1261a2d4b038c15d870...,0x148Ca723BefeA7b021C399413b8b7426A4701500


In [34]:
AUTO_LRT

AutopoolConstants(name='autoLRT', autopool_eth_addr='0xE800e3760FC20aA98c5df6A9816147f190455AF3', autopool_eth_strategy_addr='0x72a726c10220280049687E58B7b05fb03d579109', solver_rebalance_plans_bucket='v2-gen3-eth-mainnet-solve-v2gen3ethmainnetsolverau-cb9s4hq4f8sv')

In [33]:
valid_underlying_deposited_df.groupby('hash')['contract_address'].apply(list)

hash
0x00c17014c292aea3ebf6c85ff9bc694fd240ee7e77858bb2cdc18a5844197c02    [0x2f2CC1bf461413014741dD68481dB4a3686DAC3D]
0x00cdf0986c7c20722553459db4efb8fdb2eed8e941603203bf8ef5212cc320d0    [0x777FAf85c8E5FC6f4332E56B989C5C94201A273C]
0x0146005b9255612d4e769a90cedf8f6deb448cfea969e74b31fb783d6e383c0d    [0xf9779aEF9f77e78C857CB4A068c65CcBee25BAAc]
0x01614d1e3f7841bb6452cb911443987e29abca961d6cc1706e0a8bb331682de3    [0x777FAf85c8E5FC6f4332E56B989C5C94201A273C]
0x0537fc276932e5bcbb8ee28c301603da7e64f56d4257943c716fcb3f364d3009    [0x777FAf85c8E5FC6f4332E56B989C5C94201A273C]
                                                                                          ...                     
0xf06978787ae75d235cedd61c07b1e2a9037df86378f37a90eb683c87db8f98b2    [0x777FAf85c8E5FC6f4332E56B989C5C94201A273C]
0xf1a9200b85619fd39e358bf3fd8b993099e4a585735a12bfe9d4c51de8855412    [0x148Ca723BefeA7b021C399413b8b7426A4701500]
0xf20872bde7372d3106cde489b5c44c8d2af1ef0003bb233239f83c840b1708bf    [0x40

In [21]:
valid_underlying_deposited_df['hash'].value_counts()

hash
0x80d0938d7a921128c6c03a5e445601f1521cd63c49e97d1c47a6fc89c9388c43    1
0xede5866db4be0f6de1e525a9b1b0235cff05ba297f4b99f5a62f5395a85e8ac6    1
0x7ce685385887645bb3976ff021864dc504dd36428646f1432752c5215e08a012    1
0x350476010ae54e6882868908e282c799aa1f0d6c4c547e0689ecb559c72121f9    1
0xb9ffffb7aa2f8aaa490f1ab6fb371166f11c6345c4dcd84746ebcce97c63fa81    1
                                                                     ..
0xbafdc2344e3cfca7345d0f99a2340119d9a562c06e63645fa4e5c63c27e4eff5    1
0xf1a9200b85619fd39e358bf3fd8b993099e4a585735a12bfe9d4c51de8855412    1
0x3f09f5bc24ed2681ca8fe47f3e5f1510cbe6b4d808b7e8a64e201ae988f58f04    1
0x7845ffdf88a504a998b1caecf1261a2d4b038c15d870061fa9bfbd6b669cc060    1
0x4266f976892b91900ef86ff2576a235215cfa1a9abdb2f8c0288779507ddee5f    1
Name: count, Length: 114, dtype: int64

In [24]:
rebalances_we_can_compute_swap_costs_for = rebalance_between_destinations_df[rebalance_between_destinations_df['hash'].isin(valid_underlying_deposited_df['hash'])].copy()

rebalances_we_can_compute_swap_costs_for[['outDestinationVault', 'hash', 'amountOut']]

,outDestinationVault,hash,amountOut
0,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0x5a6f3a2239134ef903468d88abce973e6b71e233af04...,103.024424
1,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0x46cfba2d4f806b97e71d9465ae05556bc41cb3635c5d...,105.124100
2,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0xa0ea869df690411f31e8a430670cfaea2b69a5849715...,168.718684
3,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0xca793ddcca9f1a355bf52d55d397068f49b8c43de799...,191.505927
4,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0x6ce994c065fa58c7b58d215373dcb7480aba63965013...,144.375381
...,...,...,...
78,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0x7ea296cc9d700bc8b3a817e36f343928d997bd65d3b7...,100.000000
79,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0x06c28f98ff45d82f34a0034733773a6f10c8e80fdced...,100.000000
80,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0x84da6e052ce802cd1dc77ead5d40eb124a14c4aefc46...,99.483984
81,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0xd899eb779a909d345dac9d56abbf0a842b5fe4c5914f...,74.665379


In [ ]:
114 -

In [28]:
valid_underlying_deposited_df

,amount,sender,event,block,transaction_index,log_index,hash,contract_address
0,45275633771585782477,0xE800e3760FC20aA98c5df6A9816147f190455AF3,UnderlyingDeposited,21101723,5,75,0x80d0938d7a921128c6c03a5e445601f1521cd63c49e9...,0xC9b5D82652a1C8214b0971A004983d0EEeDD751C
1,33950989078690998560,0xE800e3760FC20aA98c5df6A9816147f190455AF3,UnderlyingDeposited,21104953,1,56,0xede5866db4be0f6de1e525a9b1b0235cff05ba297f4b...,0xC9b5D82652a1C8214b0971A004983d0EEeDD751C
2,56367497684686915345,0xE800e3760FC20aA98c5df6A9816147f190455AF3,UnderlyingDeposited,21122345,5,58,0x7ce685385887645bb3976ff021864dc504dd36428646...,0xC9b5D82652a1C8214b0971A004983d0EEeDD751C
3,42291495751400472670,0xE800e3760FC20aA98c5df6A9816147f190455AF3,UnderlyingDeposited,21125641,6,66,0x350476010ae54e6882868908e282c799aa1f0d6c4c54...,0xC9b5D82652a1C8214b0971A004983d0EEeDD751C
0,26412460747973757896,0xE800e3760FC20aA98c5df6A9816147f190455AF3,UnderlyingDeposited,20898452,0,45,0xb9ffffb7aa2f8aaa490f1ab6fb371166f11c6345c4dc...,0x90300b02b162F902B9629963830BcCCdeEd71113
...,...,...,...,...,...,...,...,...
10,93721328205063299580,0xE800e3760FC20aA98c5df6A9816147f190455AF3,UnderlyingDeposited,20889481,143,309,0xbafdc2344e3cfca7345d0f99a2340119d9a562c06e63...,0x148Ca723BefeA7b021C399413b8b7426A4701500
11,57795772975127563663,0x6dC3ce9C57b20131347FDc9089D740DAf6eB34c5,UnderlyingDeposited,20892779,38,233,0xf1a9200b85619fd39e358bf3fd8b993099e4a585735a...,0x148Ca723BefeA7b021C399413b8b7426A4701500
12,39697263089877165819,0x6dC3ce9C57b20131347FDc9089D740DAf6eB34c5,UnderlyingDeposited,20898160,2,51,0x3f09f5bc24ed2681ca8fe47f3e5f1510cbe6b4d808b7...,0x148Ca723BefeA7b021C399413b8b7426A4701500
13,74954049209195993609,0x6dC3ce9C57b20131347FDc9089D740DAf6eB34c5,UnderlyingDeposited,20912516,32,161,0x7845ffdf88a504a998b1caecf1261a2d4b038c15d870...,0x148Ca723BefeA7b021C399413b8b7426A4701500


In [ ]:
valid_underlying_deposited_df['amount']

amount
45275633771585782477    1
33950989078690998560    1
56367497684686915345    1
42291495751400472670    1
26412460747973757896    1
                       ..
93721328205063299580    1
57795772975127563663    1
39697263089877165819    1
74954049209195993609    1
68242424770291141414    1
Name: count, Length: 114, dtype: int64

In [27]:
valid_underlying_deposited_df['contract_address'].value_counts()

contract_address
0x777FAf85c8E5FC6f4332E56B989C5C94201A273C    25
0x4E12227b350E8f8fEEc41A58D36cE2fB2e2d4575    20
0xf9779aEF9f77e78C857CB4A068c65CcBee25BAAc    19
0x148Ca723BefeA7b021C399413b8b7426A4701500    15
0x40219bBda953ca811d2D0168Dc806a96b84791d9    14
0x60339056EC88996e41757E05a798310E46972cca    10
0xC9b5D82652a1C8214b0971A004983d0EEeDD751C     4
0x2F7e096a400ded5D02762120b39A3aA4ABA072a4     4
0x90300b02b162F902B9629963830BcCCdeEd71113     1
0x2f2CC1bf461413014741dD68481dB4a3686DAC3D     1
0x5c6aeb9ef0d5BbA4E6691f381003503FD0D45126     1
Name: count, dtype: int64

In [5]:
hashes_to_exclude = UnderlyingDeposited_df['hash'].value_counts()[UnderlyingDeposited_df['hash'].value_counts() > 1].index
# sunseting destiantions throws the math off and has 0 slippage anyway
hashes_to_exclude

Index(['0xbe9e2919f268121cc55f8466e7af702b6c0dd312134ee746e4600878e79efe9b',
       '0xa9b7e57b8202064b6fd48accbcfea54cb08fbb362638aa47681203c5daf7b722',
       '0x084dfee5800acd5bda5e48485e68e87c015fca770376eeef36405654a184e005'],
      dtype='object', name='hash')

In [6]:
# 0x084dfee5800acd5bda5e48485e68e87c015fca770376eeef36405654a184e005    2
# 0xbe9e2919f268121cc55f8466e7af702b6c0dd312134ee746e4600878e79efe9b    2
# 0xa9b7e57b8202064b6fd48accbcfea54cb08fbb362638aa47681203c5daf7b722    2
UnderlyingDeposited_df[UnderlyingDeposited_df['hash'] == '0xbe9e2919f268121cc55f8466e7af702b6c0dd312134ee746e4600878e79efe9b']

,amount,sender,event,block,transaction_index,log_index,hash,contract_address
0,60996771897209039619,0xE800e3760FC20aA98c5df6A9816147f190455AF3,UnderlyingDeposited,20946688,9,133,0xbe9e2919f268121cc55f8466e7af702b6c0dd312134e...,0x60339056EC88996e41757E05a798310E46972cca
1,282901426865420561721,0x6dC3ce9C57b20131347FDc9089D740DAf6eB34c5,UnderlyingDeposited,20946688,9,227,0xbe9e2919f268121cc55f8466e7af702b6c0dd312134e...,0x60339056EC88996e41757E05a798310E46972cca


In [7]:
UnderlyingWithdraw_df[UnderlyingWithdraw_df['hash'] == '0xbe9e2919f268121cc55f8466e7af702b6c0dd312134ee746e4600878e79efe9b']

,block,transaction_index,log_index,hash,contract_address,amount,owner,to,event
2,20946688,9,113,0xbe9e2919f268121cc55f8466e7af702b6c0dd312134e...,0x2F7e096a400ded5D02762120b39A3aA4ABA072a4,60996771897209039619,0xE800e3760FC20aA98c5df6A9816147f190455AF3,0x067cB94576aBBDB3527055f3c8CaAAc9668364AD,UnderlyingWithdraw
3,20946688,9,207,0xbe9e2919f268121cc55f8466e7af702b6c0dd312134e...,0x2F7e096a400ded5D02762120b39A3aA4ABA072a4,282901426865420561721,0x6dC3ce9C57b20131347FDc9089D740DAf6eB34c5,0x9cf7E7d9F02CaEdcE2286199C0c20dd92580433d,UnderlyingWithdraw


In [8]:
rebalance_between_destinations_df.columns

Index(['valueStats', 'params', 'outSummaryStats', 'inSummaryStats',
       'swapOffsetPeriod', 'predictedAnnualizedGain', 'event', 'block',
       'transaction_index', 'log_index', 'hash', 'outDestinationVault',
       'inDestinationVault', 'tokenIn', 'tokenOut', 'amountOut', 'swapCost'],
      dtype='object')

In [9]:
cols = ['hash', 'amountOut', 'outDestinationVault']
# 0xE800e3760FC20aA98c5df6A9816147f190455AF3 == autoLRT vault, all WETH
rebalance_between_destinations_df[cols]['outDestinationVault'].value_counts()

outDestinationVault
0xE800e3760FC20aA98c5df6A9816147f190455AF3    67
0x5c6aeb9ef0d5BbA4E6691f381003503FD0D45126     9
0x2F7e096a400ded5D02762120b39A3aA4ABA072a4     3
0x777FAf85c8E5FC6f4332E56B989C5C94201A273C     2
0x148Ca723BefeA7b021C399413b8b7426A4701500     1
0x4E12227b350E8f8fEEc41A58D36cE2fB2e2d4575     1
Name: count, dtype: int64

In [10]:
UnderlyingDeposited_df['hash'].value_counts()

hash
0xbe9e2919f268121cc55f8466e7af702b6c0dd312134ee746e4600878e79efe9b    2
0xa9b7e57b8202064b6fd48accbcfea54cb08fbb362638aa47681203c5daf7b722    2
0x084dfee5800acd5bda5e48485e68e87c015fca770376eeef36405654a184e005    2
0xede5866db4be0f6de1e525a9b1b0235cff05ba297f4b99f5a62f5395a85e8ac6    1
0x80d0938d7a921128c6c03a5e445601f1521cd63c49e97d1c47a6fc89c9388c43    1
                                                                     ..
0xbafdc2344e3cfca7345d0f99a2340119d9a562c06e63645fa4e5c63c27e4eff5    1
0xf1a9200b85619fd39e358bf3fd8b993099e4a585735a12bfe9d4c51de8855412    1
0x3f09f5bc24ed2681ca8fe47f3e5f1510cbe6b4d808b7e8a64e201ae988f58f04    1
0x7845ffdf88a504a998b1caecf1261a2d4b038c15d870061fa9bfbd6b669cc060    1
0x4266f976892b91900ef86ff2576a235215cfa1a9abdb2f8c0288779507ddee5f    1
Name: count, Length: 117, dtype: int64

In [11]:
# amount 

In [12]:
rebalance_between_destinations_df['hash'].value_counts()

hash
0x5a6f3a2239134ef903468d88abce973e6b71e233af04ac0ee42651291e045294    1
0x46cfba2d4f806b97e71d9465ae05556bc41cb3635c5d133c421271a09b91de16    1
0xa0ea869df690411f31e8a430670cfaea2b69a584971567945e5d38b32d89f062    1
0xca793ddcca9f1a355bf52d55d397068f49b8c43de7998f804831daa3e9d48574    1
0x6ce994c065fa58c7b58d215373dcb7480aba639650136214a4a17ee64a3e595a    1
                                                                     ..
0x7ea296cc9d700bc8b3a817e36f343928d997bd65d3b77427bfd4ce725e82d35a    1
0x06c28f98ff45d82f34a0034733773a6f10c8e80fdcedaf91ef0f08dfd8ee35ee    1
0x84da6e052ce802cd1dc77ead5d40eb124a14c4aefc467f44df1d707cef83b900    1
0xd899eb779a909d345dac9d56abbf0a842b5fe4c5914f7058f8cb92f9c7f842d8    1
0x9aa6839667424f557b48ce54eac0916b8590810d4b9136624700dd2e57e12e57    1
Name: count, Length: 83, dtype: int64

In [13]:
RebalanceBetweenDestinations, RebalanceToIdle, SuccessfulRebalanceBetweenDestinations = dfs
RebalanceBetweenDestinations.shape, RebalanceToIdle.shape, SuccessfulRebalanceBetweenDestinations.shape

NameError: name 'dfs' is not defined

In [ ]:
RebalanceToIdle["tokenIn"] = RebalanceToIdle["params"].apply(
    lambda x: eth_client.toChecksumAddress(x[1])
)

RebalanceToIdle["tokenOut"] = RebalanceToIdle["params"].apply(
    lambda x: eth_client.toChecksumAddress(x[4])
)

RebalanceToIdle["amountOut"] = RebalanceToIdle["params"].apply(
    lambda x: x[5]
    / 1e18  # the amout of lp tokens taken out is always correct, what we don't know is the amount of lp tokens going in
    )
RebalanceToIdle

In [ ]:
RebalanceToIdle['hash'].values

In [ ]:
RebalanceToIdle['hash'].isin(RebalanceBetweenDestinations['hash'])

In [ ]:
SuccessfulRebalanceBetweenDestinations['hash'].isin(RebalanceBetweenDestinations['hash'])

In [ ]:
0x084dfee5800acd5bda5e48485e68e87c015fca770376eeef36405654a184e005    2
0xbe9e2919f268121cc55f8466e7af702b6c0dd312134ee746e4600878e79efe9b    2
0xa9b7e57b8202064b6fd48accbcfea54cb08fbb362638aa47681203c5daf7b722    2
UnderlyingDeposited_df[UnderlyingDeposited_df['hash'] == '0xbe9e2919f268121cc55f8466e7af702b6c0dd312134ee746e4600878e79efe9b']

In [ ]:
UnderlyingDeposited_df[UnderlyingDeposited_df['hash'] == '0xbe9e2919f268121cc55f8466e7af702b6c0dd312134ee746e4600878e79efe9b']

In [ ]:
rebalance_between_destinations_df[rebalance_between_destinations_df['hash'] == '0xbe9e2919f268121cc55f8466e7af702b6c0dd312134ee746e4600878e79efe9b']

In [ ]:
UnderlyingDeposited_df['hash'].value_counts()

In [ ]:
def underlying_deposited_df_to_change(row):
    return {
        'Destination': row['contract_address'],
        'lpTokenQuantity': row['amount'] /1e18,
        'hash':row['hash'],
        'block': int(row['block'])
    }

deposit_df = pd.DataFrame.from_records(UnderlyingDeposited_df.apply(underlying_deposited_df_to_change, axis=1).values)

def underlying_withdrawn_df_to_change(row):
    return {
        'Destination': row['contract_address'],
        'lpTokenQuantity':  - row['amount'] /1e18,
        'hash':row['hash'],
        'block': int(row['block'])
    }

withdraw_df = pd.DataFrame.from_records(UnderlyingWithdraw_df.apply(underlying_withdrawn_df_to_change, axis=1).values)
change_df = pd.concat([deposit_df, withdraw_df]).sort_values('block')
change_df

In [ ]:
# AMOUNT_OUT IS THE QUANITY OF LP TOKENS THAT LEFT
# WE JUST NEED THE DEPOSTED

In [ ]:
df = pd.merge(change_df, validated_spot_price_df, on='block')

def _change_in_spot_value(row):
    spot_price =  row[row['Destination']]
    change = spot_price * row['lpTokenQuantity']
    return change

df['spot_value_change'] = df.apply(_change_in_spot_value, axis=1)

a = df.groupby('hash')['spot_value_change'].apply(list)

between_destination_hashes = a[a.apply(len) == 2].index
from_idle_hashes = a[a.apply(len) == 1].index

In [ ]:
amounts_and_quantities = pd.merge(withdraw_df[['hash', 'lpTokenQuantity']], rebalance_between_destinations_df[['amountOut', 'hash']], on='hash')
amounts_and_quantities[amounts_and_quantities['hash'].isin(between_destination_hashes)]

In [ ]:
amounts_and_quantities = pd.merge(df[['hash', 'lpTokenQuantity']], rebalance_between_destinations_df[['amountOut', 'hash']], on='hash')
amounts_and_quantities[amounts_and_quantities['hash'].isin(from_idle_hashes)]

In [ ]:
rebalance_between_destinations_df[['amountOut', 'hash']]

In [ ]:
rebalance_between_destinations_df[rebalance_between_destinations_df['hash'].isin(between_destination_hashes)][['outDestinationVault', 'inDestinationVault']].value_counts()

In [ ]:
df.groupby('hash')['spot_value_change'].apply(len) == 2

In [ ]:
UnderlyingWithdraw_df

In [ ]:


# UnderlyingDeposited_df["amountDeposited"] = UnderlyingDeposited_df["amount"] / 1e18
# UnderlyingWithdraw_df["amountWithdraw"] = UnderlyingWithdraw_df["amount"] / 1e18

# df = pd.merge(rebalance_between_destinations_df, UnderlyingDeposited_df[['hash', 'amountDeposited']], how='left', on='hash')
# df = pd.merge(df, UnderlyingWithdraw_df[['hash', 'amountWithdraw']], how='left', on='hash')
# cols = ['hash', 'amountWithdraw', 'amountDeposited', 'amountOut', 'tokenIn', 'tokenOut']

In [ ]:
# df['hash'].value_counts() # exclude the funny ones, with destinations getting added or withdrawn

In [ ]:
df[df["hash"] == "0x084dfee5800acd5bda5e48485e68e87c015fca770376eeef36405654a184e005"][cols]

In [ ]:
UnderlyingDeposited_df[
    UnderlyingDeposited_df["hash"] == "0x084dfee5800acd5bda5e48485e68e87c015fca770376eeef36405654a184e005"
]

In [ ]:
UnderlyingWithdraw_df[
    UnderlyingWithdraw_df["hash"] == "0x084dfee5800acd5bda5e48485e68e87c015fca770376eeef36405654a184e005"
]

In [ ]:
df2 = df.dropna()
df2

In [ ]:
WETH = "0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2"
#  emit UnderlyingDeposited(amount, msg.sender);

In [ ]:
rebalance_between_destinations_df[["tokenIn", "tokenOut", "amountOut", "destinationIn"]]

In [ ]:
rebalance_between_destinations_df["hash"].values

In [ ]:
first_out = "0xE800e3760FC20aA98c5df6A9816147f190455AF3"
first_in = "0x777FAf85c8E5FC6f4332E56B989C5C94201A273C"
balance_of_undelrying_debt_before[[first_in, first_out]]
# I can't check the balances before or after

In [ ]:
balance_of_undelrying_debt_before[[first_in, first_out]]

In [ ]:
balance_of_undelrying_debt_after[[first_in, first_out]]

In [ ]:
rebalance_between_destinations_df

In [ ]:
rebalance_between_destinations_df

In [ ]:
validated_spot_price_df

In [ ]:
# from mainnet_launch.autopool_diagnostics.compute_rebalance_cost import (
#     compute_daily_rebalance_costs,
#     AUTO_LRT, BAL_ETH, AUTO_ETH,
#     AutopoolConstants,
# )
# import pandas as pd
# import plotly.express as px


# # # return RebalanceValueStats({
# # #     inPrice: inPrice,
# # #     outPrice: outPrice,
# # #     inEthValue: inEthValue,
# # #     outEthValue: outEthValue,
# # #     swapCost: swapCost,
# # #     slippage: slippage
# # # });


# # /// @param destinationIn The address / lp token of the destination vault that will increase
# # /// @param tokenIn The address of the underlyer token that will be provided by the swapper
# # /// @param amountIn The amount of the underlying LP tokens that will be received
# # /// @param destinationOut The address of the destination vault that will decrease
# # /// @param tokenOut The address of the underlyer token that will be received by the swapper
# # /// @param amountOut The amount of the tokenOut that will be received by the swapper
# # struct RebalanceParams {
# #     address destinationIn;
# #     address tokenIn;
# #     uint256 amountIn;
# #     address destinationOut;
# #     address tokenOut;
# #     uint256 amountOut;
# # }
# autopool = AUTO_ETH
# a, b = compute_daily_rebalance_costs(autopool)

In [ ]:
autopool = AUTO_ETH

In [ ]:
import plotly.io as pio
from mainnet_launch.destinations import get_destination_details, ALL_AUTOPOOLS

pio.templates.default = None
rebalance_from_idle, transfer_events_df, validated_spot_price_df = b
rebalance_between_destinations, transfer_events_df, validated_spot_price_df = a

In [ ]:
desination_vault_addresses = [d.vaultAddress for d in get_destination_details() if d.autopool == autopool]
desination_vault_addresses

In [ ]:
def _get_limited_transfers(
    autopool: AutopoolConstants, transfer_events_df: pd.DataFrame, rebalance_df: pd.DataFrame
) -> pd.DataFrame:
    desination_vault_addresses = [d.vaultAddress for d in get_destination_details() if d.autopool == autopool]
    sender_or_receiver_is_destination_vault = (transfer_events_df["to"].isin(desination_vault_addresses)) | (
        transfer_events_df["from"].isin(desination_vault_addresses)
    )
    destination_lp_token_addresses = [d.vaultAddress for d in get_destination_details() if d.autopool == autopool]
    sent_token_is_lp_token_avaiable_to_autopool = transfer_events_df["token_address"].isin(
        destination_lp_token_addresses
    )

    tx_hash_in_rebalance_df_hash = transfer_events_df["hash"].isin(rebalance_df["hash"])

    limited_transfers = (
        transfer_events_df[
            sender_or_receiver_is_destination_vault
            & sent_token_is_lp_token_avaiable_to_autopool
            & tx_hash_in_rebalance_df_hash
        ]
        .copy()
        .reset_index()
    )

    return limited_transfers


limited_transfers = _get_limited_transfers(autopool, transfer_events_df, rebalance_between_destinations)
limited_transfers["hash"].value_counts()

In [ ]:
# def compute_swap_cost_when_rebalance_from_idle(
#     rebalance_from_idle_df: pd.DataFrame, autopool: AutopoolConstants, transfer_events_df: pd.DataFrame, validated_spot_price_df:pd.DataFrame
# ) -> pd.DataFrame:
#     # out means decrease
#     # in means increase
#     if (rebalance_from_idle_df["outDestinationVault"] != autopool.autopool_eth_addr).any():
#         raise ValueError(
#             "trying to compute rebalance cost on rebalance_from_idle_df when the outDestinationVault != autopool "
#         )


#     #

#     transfers_to_vaults = (
#         transfers_to_or_from_autopool_destinations[
#             (transfer_events_df["hash"].isin(rebalance_from_idle_df["hash"]))
#             & (transfer_events_df["to"].isin(rebalance_from_idle_df["inDestinationVault"]))
#         ]

#         .drop_duplicates()
#         .copy()
#     )

#     rebalance_from_idle_with_lp_tokens_in = pd.merge(
#         rebalance_from_idle_df, transfers_to_vaults[["value", "hash"]], on="hash"
#     )

#     rebalance_from_idle_with_lp_tokens_in["lp_tokens_added_to_vault"] = (
#         rebalance_from_idle_with_lp_tokens_in["value"]/ 1e18
#     )

#     validated_spot_price_df["block_with_rebalance"] = ( validated_spot_price_df["block"] + 1)

#     rebalance_from_idle_df = pd.merge(
#         validated_spot_price_df,
#         rebalance_from_idle_with_lp_tokens_in,
#         right_on="block",
#         left_on="block_with_rebalance",
#     )

#     def _compute_spot_value_added_to_vault(row:dict) -> float:
#         return row['lp_tokens_added_to_vault'] * row[row['inDestinationVault']] # the pool itsel

#     rebalance_from_idle_df['validated_spot_value_of_lp_tokens_added_to_autopool'] = rebalance_from_idle_df.apply(_compute_spot_value_added_to_vault, axis=1)
#     # because we are exiting idle, the value out is in WETH so the spot value is always 1
#     rebalance_from_idle_df['validated_spot_value_of_lp_tokens_removed_from_autopool'] = rebalance_from_idle_df["params_amountOut"]
#     rebalance_from_idle_df['validated_spot_value_of_swap_cost'] = rebalance_from_idle_df['validated_spot_value_of_lp_tokens_removed_from_autopool'] -  rebalance_from_idle_df['validated_spot_value_of_lp_tokens_added_to_autopool']

#     rebalance_from_idle_df['validated_spot_value_of_swap_cost'] = rebalance_from_idle_df['validated_spot_value_of_swap_cost'].clip(lower=0)

#     rebalance_from_idle_df['swap_cost_from_event'] = rebalance_from_idle_df['valueStats'].apply(lambda x: float(x[4]) /1e18)

#     # not sure what do to with the swap cost
#     rebalance_from_idle_df.index = validated_spot_price_df.index
#     cumulative_daily_validated_swap_cost =  rebalance_from_idle_df[['validated_spot_value_of_swap_cost']].resample("1D").sum().cumsum()
#     return cumulative_daily_validated_swap_cost

# rebalance_events_df, transfer_events_df, validated_spot_price_df = b
# cumulative_daily_validated_swap_cost = compute_swap_cost_when_rebalance_from_idle(
#     rebalance_events_df.copy(), AUTO_ETH, transfer_events_df, validated_spot_price_df
# )

# cumulative_daily_validated_swap_cost

In [ ]:
# rebalance_between_destinations_df, transfer_events_df, validated_spot_price_df = a

# def _extract_relevent_transfers(autopool: AutopoolConstants, transfer_events_df: pd.DataFrame, rebalance_between_destinations_df:pd.DataFrame):

#     valid_destinations = [d.vaultAddress for d in get_destination_details() if d.autopool == autopool]
#     valid_transfer_events = transfer_events_df[(transfer_events_df['to'].str.lower().isin(valid_destinations)) | (transfer_events_df['from'].str.lower().isin(valid_destinations))]

#     to_indest = (
#         transfer_events_df[
#             (transfer_events_df["hash"].isin(rebalance_between_destinations_df["hash"]))
#             & (transfer_events_df["to"].isin(valid_destinations))
#         ]
#         .drop_duplicates()
#         .copy()
#     )

#     to_outdest = (
#         transfer_events_df[
#             (transfer_events_df["hash"].isin(rebalance_between_destinations_df["hash"]))
#             & (transfer_events_df["to"].isin(valid_destinations))
#         ]

#         .drop_duplicates()
#         .copy()
#     )


#     from_indest = (
#         transfer_events_df[
#             (transfer_events_df["hash"].isin(rebalance_between_destinations_df["hash"]))
#             & (transfer_events_df["from"].isin(valid_destinations))
#         ]
#         .drop_duplicates()
#         .copy()
#     )

#     from_outdest = (
#         transfer_events_df[
#             (transfer_events_df["hash"].isin(rebalance_between_destinations_df["hash"]))
#             & (transfer_events_df["from"].isin(valid_destinations))
#         ]

#         .drop_duplicates()
#         .copy()
#     )


#     return to_indest, to_outdest, from_indest, from_outdest

# to_indest, to_outdest, from_indest, from_outdest = _extract_relevent_transfers(AUTO_ETH, transfer_events_df, rebalance_between_destinations_df)
# to_indest.shape, to_outdest.shape, from_indest.shape, from_indest.shape,

In [ ]:
between_with_from_indest = pd.merge(rebalance_between_destinations_df, from_indest[["value", "hash"]], on="hash")

between_with_from_indest["hash"].value_counts()

In [ ]:
rebalance_between_destinations_df[
    rebalance_between_destinations_df["hash"] == "0x5a088e71e51c5b8c05db282989f21975d14e44d9c615aa00e451a68dcbebd0dc"
]

In [ ]:
# 0x5a088e71e51c5b8c05db282989f21975d14e44d9c615aa00e451a68dcbebd0dc rebalances more than one block

In [ ]:
transfers_from_vaults[
    transfers_from_vaults["hash"] == "0x5a088e71e51c5b8c05db282989f21975d14e44d9c615aa00e451a68dcbebd0dc"
]

In [ ]:
transfers_from_vaults["hash"].value_counts()

In [ ]:
# # def compute_swap_cost_when_rebalancing_between_destinations(
# #     rebalance_not_from_idle_df: pd.DataFrame, autopool: AutopoolConstants, transfer_events_df: pd.DataFrame, validated_spot_price_df:pd.DataFrame
# # ) -> pd.DataFrame:
# #     # out means decrease
# #     # in means increase
# #     if (rebalance_not_from_idle_df["outDestinationVault"] == autopool.autopool_eth_addr).any():
# #         raise ValueError(
# #             "trying to compute_swap_cost_when_rebalancing_between_destinations when the outDestinationVault == autopool eg, exiting the autopool "
# #         )


# #     limited_transfer_df = (
# #         transfer_events_df[
# #             (transfer_events_df["hash"].isin(rebalance_not_from_idle_df["hash"]))
# #         ]

# #         .drop_duplicates()
# #         .copy()
# #     )
# #     return limited_transfer_df


# #

# # limited_transfer_df = compute_swap_cost_when_rebalancing_between_destinations(
# #     rebalance_events_df.copy(), AUTO_ETH, transfer_events_df, validated_spot_price_df
# # )

# # transfers_to_vaults['to_value'] = transfers_to_vaults['value'] /1e18
# # transfers_to_vaults['to_token'] = transfers_to_vaults['token_address']

# # transfers_from_vaults['from_value'] = transfers_from_vaults['value'] /1e18
# # transfers_from_vaults['from_token'] = transfers_from_vaults['token_address']

# def build_lp_token_moved_df(row):

#     return {
#         'DesinationIncreased': row['to_token'],
#         'DestinationDecreased': row['from_token'],
#         'amountIncreased': row['to_value'],
#         'amountDecreased': row['from_value'],
#         'hash':row['hash'],
#         'block': row['block']
#     }

# limited_transfer_df['hash'].value_counts()

In [ ]:
rebalance_events_df

In [ ]:
transfers_from_vaults

In [ ]:
rebalance_events_df["inDestinationVault"].value_counts()

In [ ]:
rebalance_events_df["outDestinationVault"].value_counts()

In [ ]:
AUTO_ETH.autopool_eth_addr

In [ ]:
rebalance_from_idle_df.sort_values("rebalance_cost")[["hash", "rebalance_cost"]].values[0]

0xf06978787ae75d235cedd61c07b1e2a9037df86378f37a90eb683c87db8f98b2

This rebalance, my math thinks that

In [ ]:
rebalance_from_idle_df["valueStats_swap_cost"] = rebalance_from_idle_df["valueStats"].apply(lambda x: x[4] / 1e18)
px.line(rebalance_from_idle_df[["rebalance_cost", "valueStats_swap_cost"]])

In [ ]:
dest_to_dest = lp_token_transfer_df[(lp_token_transfer_df["hash"] == tx_hash)].copy().drop_duplicates()
dest_to_dest

In [ ]:
# 165151197070039380229 expect outDestinationLpToken to be down

today I am working more on fixing the overestimate the rebalancing cost on the dashboard. 

In [ ]:
# idle -> destination valueStats.amountOut == quantiy of eth that left
#

In [ ]:
rebalance_events_df.shape

In [ ]:
160795862717038788608 + 419684887152871954

In [ ]:
dest_to_dest[dest_to_dest["token_address"] == inDestinationVault]

In [ ]:
dest_to_dest[dest_to_dest["token_address"] == outDestinationVault]

In [ ]:
dest_to_dest[dest_to_dest["to"] == outDestinationVault]

In [ ]:
dest_to_dest[dest_to_dest["from"] == outDestinationVault]
# destination vault -> flash borrow solver 0x067cB94576aBBDB3527055f3c8CaAAc9668364AD
# is the quanikty t

In [ ]:
dest_to_dest["to"].value_counts()

In [ ]:
rebalance_events_df.values[-1]  # 73804358107532979104

In [ ]:
rebalance_events_df